#### For libraries install. 
If something is missing (given the code) add here the pip install

In [ ]:
# for libraries install
!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain

In [1]:
import igraph as ig
import louvain

##### Load first net state for all types of connection

In [2]:
G = ig.Graph()
G = G.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml") 
print(G)

IGRAPH D--- 6106 58733 --
+ edges:
90->780 309->14 961->1042 1267->1114 364->733 1307->2 195->37 2->755 660->522
797->913 2->3 644->643 339->360 5->4 536->247 813->814 165->36 723->714
296->172 981->754 141->597 394->393 329->268 422->38 280->69 829->492
229->1089 206->1252 5->97 1088->364 763->163 90->780 12->234 258->322 432->62
521->696 4->322 1211->589 465->266 229->1088 260->259 5->4 797->816 371->372
817->482 46->47 1190->268 1034->1095 278->1281 271->365 226->352 522->827
578->497 116->168 643->765 867->755 272->237 534->328 245->62 907->11
158->1272 364->928 76->118 1088->193 90->780 641->1094 829->320 271->112
424->425 144->568 463->920 9->978 89->90 963->964 539->62 565->165 507->296
627->628 320->48 21->476 763->775 841->200 418->417 144->268 674->617 148->276
294->792 273->193 631->719 685->38 285->431 763->874 78->308 210->95 1200->294
1089->229 418->417 859->490 903->907 741->682 437->268 8->167 776->178
816->104 365->578 179->180 422->38 1252->625 1060->139 822->7 755->2

In [12]:
# print(G.vs.attribute_names())
# print(G.es.attribute_names())
# print(G.degree())
# print(G.indegree())
# print(G.outdegree())

[11, 56, 19, 10, 33, 47, 15, 12, 16, 42, 24, 7, 14, 8, 2, 12, 1, 7, 14, 7, 6, 11, 7, 8, 14, 3, 10, 12, 13, 14, 29, 10, 6, 26, 1, 24, 7, 36, 104, 39, 5, 0, 8, 3, 7, 3, 3, 110, 11, 1, 7, 2, 3, 15, 6, 13, 3, 0, 1, 8, 9, 8, 66, 0, 7, 5, 5, 2, 34, 25, 31, 19, 14, 4, 4, 18, 10, 0, 11, 15, 18, 5, 2, 34, 19, 4, 1, 4, 8, 2, 132, 16, 6, 10, 5, 11, 9, 15, 1, 6, 39, 0, 25, 0, 6, 17, 10, 38, 1, 5, 5, 7, 18, 0, 2, 3, 8, 7, 65, 7, 45, 4, 33, 21, 1, 5, 5, 2, 12, 1, 3, 0, 9, 0, 3, 0, 12, 1, 5, 5, 4, 6, 1, 3, 46, 5, 17, 0, 7, 0, 5, 8, 34, 2, 0, 9, 2, 2, 9, 11, 2, 0, 15, 28, 5, 74, 2, 38, 19, 3, 2, 2, 20, 31, 3, 2, 11, 2, 18, 6, 8, 5, 13, 19, 0, 4, 30, 0, 11, 0, 25, 30, 0, 57, 2, 38, 1, 3, 0, 7, 10, 1, 22, 6, 5, 2, 4, 1, 3, 9, 49, 19, 0, 55, 5, 6, 0, 2, 1, 10, 0, 1, 4, 9, 5, 10, 4, 6, 0, 65, 13, 27, 1, 24, 10, 10, 0, 41, 1, 3, 1, 12, 20, 6, 13, 1, 3, 16, 8, 6, 3, 0, 12, 2, 3, 2, 0, 2, 39, 4, 12, 2, 15, 2, 2, 10, 17, 2, 90, 13, 8, 24, 10, 3, 2, 4, 23, 21, 6, 3, 14, 0, 0, 27, 1, 18, 22, 5, 35, 6, 7, 8, 1, 